First Tensorflow demo

In [3]:
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import time


import tensorflow as  tf

In [19]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Exampple tried is evaluating $f(x,y) = x^2 y + y + 2$

### Creating and running a graph

In [5]:
import tensorflow as tf

In [21]:
#create variables and default graph
reset_graph()

#Variables and constants
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
a = tf.constant(2, name="a")

In [22]:
# are these nodes on the graph
print(x.graph is tf.get_default_graph())
print(y.graph is tf.get_default_graph())
print(a.graph is tf.get_default_graph())

True
True
True


In [23]:
#What exactly is x, y, a
for i in [x,y,a]:
    print("")
    print(i)
    print(type(i))


<tf.Variable 'x:0' shape=() dtype=int32_ref>
<class 'tensorflow.python.ops.variables.Variable'>

<tf.Variable 'y:0' shape=() dtype=int32_ref>
<class 'tensorflow.python.ops.variables.Variable'>

Tensor("a:0", shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>


In [25]:
#define function f(x,y)


f = x * x * y + y + a

In [26]:
#Lazy evaluation : As of now only graph has been created
f

<tf.Tensor 'add_3:0' shape=() dtype=int32>

#### For evaluating we create a Tensorflow session

A TF sessiono initializes all variables and evaluates the grapth. It pits grapth oprtatiob on a CPU or GPU(or a cluster) and holds all the variable values

In [27]:
# A Verbose way:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

a_val = a.eval(session=sess) # A constant needs not be initialized
print('a = ', a_val)
print('result = ', result)
sess.close()

a =  2
result =  42


In [28]:
# Run a session with automatic closing at the end
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()  #
    a_val = a.eval()
    
print('a = ', a_val)
print('result = ', result)

a =  2
result =  42


#### Initialization of all variables at once

In [29]:
init = tf.global_variables_initializer() #prepare an init node on the graph

with tf.Session() as sess:
    init.run() #initialize all the variables
    result = f.eval()
    
print(result)

42


In [30]:
init

<tf.Operation 'init' type=NoOp>

#### Lifecycle of a node value

In [31]:
w = tf.constant(3)

x = w + 2
y = x + 5
z = x * 3

#int this example, code evaluates w ad x twice:
with tf.Session() as sess:
    print(y.eval()) #10
    print(z.eval()) #15
    


10
15


All nodes values are dropped between runs, except variable values

In [32]:
x

<tf.Tensor 'add_4:0' shape=() dtype=int32>

#### A more efficient evaluation of the Tensorflow graph

In [34]:
# in this example, the evaluation of w and x only done once:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val) #10
    print(z_val) #15

10
15


#### Reverse-mode autodiff in TensorFlow

Define a composite function $f(w)=exp(w_{20} + w_{21} * exp(w_{10} + w_{11}*exp(w_{00}+w_{01} * x)))$

In [35]:
#Implement this in Tensor Flow

def my_func(w,x):
    f_0 = tf.exp(w[0,0] + w[0,1]*x)   #the iiner-most function
    f_1 = tf.exp(w[1,0] + w[1,1]*f_0) #the next-level function
    f_2 = tf.exp(w[2,0] + w[2,1]*f_1) #the output function
    return f_2, f_1, f_0


In [45]:
#fancier implementation using name scopes:

def my_func(w,x):
    with tf.name_scope("f_0_level") as scope_0:
        f_0 = tf.exp(w[0,0] + w[0,1]*x)
    with tf.name_scope("f_1_level") as scope_1:
        f_1 = tf.exp(w[1,0] + w[1,1]*f_0)
    with tf.name_scope("f_2_level") as scope_2:
        f_2 = tf.exp(w[2,0] + w[2,1]*f_1)
    return f_2, f_1, f_0

In [46]:
#w_0 is a point at which we want to compute the function and its derivatives

w_0 = np.vstack((np.zeros(3), np.ones(3))).T
w_0

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

#### Compute gradients using TF

In [50]:
reset_graph()


#Variables and their initialization
w = tf.Variable(w_0, name="w", dtype=tf.float32)
x = tf.Variable(1.0, name="x", dtype=tf.float32, trainable=False)

#Define nodes for values of the function at layers
f_2, f_1, f_0 = my_func(w,x)

#Define nodes for th gradients by Tensorflow's autodiff
grads = tf.gradients(f_2, w)

grads

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.Session object at 0x11c1d0978>>
Traceback (most recent call last):
  File "/Users/jvsingh/anaconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 579, in __del__
    if self._session is not None:
AttributeError: 'Session' object has no attribute '_session'


[<tf.Tensor 'gradients/AddN:0' shape=(3, 2) dtype=float32>]

In [53]:
#Another way

#A node for initializer
init = tf.global_variables_initializer()

#Run the session
t_0 = time.time()

with tf.Session() as sess:
    sess.run(init)
    
    #function_vals = sess.run([f_2, f_1, f_0])
    #gradients     = sess.run(grads)
    gradients, function_vals = sess.run([grads, [f_2, f_1, f_0]])
    

print("Computed derivatives in %f3.2 sec" % (time.time() - t_0))
print("Function values = ", function_vals)
print("Gradient = ", gradients)

Computed derivatives in 0.0189193.2 sec
Function values =  [3814273.0, 15.154261, 2.7182817]
Gradient =  [array([[  1.57123440e+08,   1.57123440e+08],
       [  5.78024880e+07,   1.57123440e+08],
       [  3.81427300e+06,   5.78024880e+07]], dtype=float32)]
